# Using Python for visualizing neuroimaging data

The primary goal of this workshop is to become familiar with loading, modifying, saving, and visualizing neuroimages in Python. A secondary goal is to develop a conceptual understanding of the data structures involved, to facilitate diagnosing problems in data or analysis pipelines.

To these ends, we'll be exploring two libraries: [nibabel](http://nipy.org/nibabel/) and [nilearn](https://nilearn.github.io/).

Each of these projects has excellent documentation. While this should get you started, it is well worth your time to look through these sites.

From [1](https://nbviewer.jupyter.org/github/nipy/workshops/blob/master/170327-nipype/notebooks/nibabel_nilearn/nibabel_nilearn.ipynb)

## Setup

In [ ]:
import sys
import os
import numpy as np
import nibabel as nib
import nilearn as nl

# Image settings
import nilearn.plotting
import matplotlib as mpl
import pylab as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline
mpl.rcParams['font.family'] = 'DejaVu Sans'

out_dir = '/tmp'

Just to confirm we're all on the same page, your versions should be the same as mine:

In [ ]:
print("Python version:", sys.version.split()[0])
print("nibabel version: {} (Commit: {})".format(nib.__version__, open(os.path.join(os.path.dirname(nib.__file__), 'COMMIT_INFO.txt')).readlines()[1].split()[2]))
print("nilearn version:", nilearn.__version__)

# Nibabel

Nibabel is a low-level Python library that gives access to a variety of imaging formats, with a particular focus on providing a common interface to the various volumetric formats produced by scanners and used in common neuroimaging toolkits.

 - NIfTI-1
 - NIfTI-2
 - SPM Analyze
 - FreeSurfer .mgh/.mgz files
 - Philips PAR/REC
 - Siemens ECAT
 - DICOM (limited support)

It also supports surface file formats

 - GIFTI
 - FreeSurfer surfaces, labels and annotations

Connectivity

 - CIFTI-2 (next release! but included in this container)

Tractocgraphy

 - TrackViz .trk files

And a number of related formats.

*Almost* all of these can be loaded through the `nibabel.load` interface.

## Loading and inspecting images in nibabel

In [ ]:
t1 = nib.load('/data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz')
print(t1)

This data-affine-header structure is common to volumetric formats in nibabel, though the details of the header will vary from format to format.

These objects can be accessed through the following interfaces.

In [ ]:
data = t1.get_data()
affine = t1.affine
header = t1.header

###### Aside
Why not just `t1.data`? Working with neuroimages can use a lot of memory, so nibabel works hard to be memory efficient. If it can read some data while leaving the rest on disk, it will. `t1.get_data()` reflects that it's doing some work behind the scenes.

### Data

The data is a simple numpy array. It has a shape, it can be sliced and generally manipulated as you would any array.

In [ ]:
print(data.shape)
fig = plt.imshow(data[:, :, data.shape[2] // 2].T, cmap='Greys_r')
fig.axes.set_xticks([])
_ = fig.axes.set_yticks([])

## Exercise 1:

Load the T1 data from subject 2. Plot the image using the same volume indexing as was done for subject 1. Also print the shape of the data.

In [ ]:
t1_2 = nib.load('/data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz')
data_2 = t1_2.get_data()
fig = plt.imshow(data_2[:, :, data_2.shape[2] // 2].T, cmap='Greys_r')
fig.axes.set_xticks([])
_ = fig.axes.set_yticks([])
print(data_2.shape)

In [ ]:
# Work on solution here

Nibabel has its own viewer, which can be accessed through `img.orthoview()`. This viewer scales voxels to reflect their size, and labels orientations. From a normal IPython console, it will create an interactive window, and you can click to select different slices, similar to `mricron`.

In [ ]:
t1.orthoview()

###### Quirk

 - `img.get_data_dtype()` shows the type of the data on disk
 - `img.get_data().dtype` shows the type of the data that you're working with

These are not always the same, and not being clear on this [has caused problems](https://github.com/nipy/nibabel/issues/490). Further, modifying one does not update the other. This is especially important to keep in mind later, when saving files.

In [ ]:
print((data.dtype, t1.get_data_dtype()))

###### Warning

`img.orthoview()` may not work properly on OS X.

### Affine

The affine is a 4 x 4 numpy array. This describes the transformation from the voxel space (indices [i, j, k]) to the reference space (distance in mm (x, y, z)).

It can be used, for instance, to discover the voxel that contains the origin of the image:

In [ ]:
x, y, z, _ = np.linalg.pinv(affine).dot(np.array([0, 0, 0, 1])).astype(int)

print("Affine:")
print(affine)
print
print("Center: ({:d}, {:d}, {:d})".format(x, y, z))

The affine also encodes the axis orientation and voxel sizes:

In [ ]:
nib.aff2axcodes(affine)

In [ ]:
nib.affines.voxel_sizes(affine)

## Exercise 2:

For subject 2:

- Determine the center voxel 
- Determine axis orientation
- Print voxel sizes
- Display using orthoview

In [ ]:
t1_2 = nib.load('/data/ds000114/sub-02/ses-test/anat/sub-02_ses-test_T1w.nii.gz')
affine_2 = t1_2.affine

x, y, z, _ = np.linalg.pinv(affine_2).dot(np.array([0, 0, 0, 1])).astype(int)

print("Affine:")
print(affine_2)
print
print("Center: ({:d}, {:d}, {:d})".format(x, y, z))

In [ ]:
nib.aff2axcodes(affine_2)

In [ ]:
nib.affines.voxel_sizes(affine_2)

In [ ]:
t1_2.orthoview()

In [ ]:
# Determine center

In [ ]:
# Determine axis orientation

In [ ]:
# Print voxel sizes

In [ ]:
# Display using orthoview

### Header

The header is a nibabel structure that stores all of the metadata of the image. You can query it directly, if necessary:

In [ ]:
print(header)
print
header.structarr['descrip']

But it also provides interfaces for the more common information.

In [ ]:
for method in ('get_zooms', 'get_xyzt_units', 'get_qform', 'get_sform'):
    print("header.{}()".format(method))
    print(getattr(header, method)())
    print

Normally, we're not particularly interested in the header or the affine. But it's important to know they're there, and, especially, to remember to copy them when making new images, so that derivatives stay aligned with the original image.

## nib-ls

Nibabel comes packaged with a command-line tool to print common metadata about any (volumetric) neuroimaging format nibabel supports. By default, it shows (on-disk) data type, dimensions and voxel sizes. We can also inspect header fields by name, for instance, `descrip`:

In [ ]:
!nib-ls -H descrip /data/ds000114/sub-01/ses-test/anat/sub-01_ses-test_T1w.nii.gz

## Creating and saving images

Suppose we want to save space by rescaling our image to a smaller datatype, such as an unsigned byte. To do this, we create a new image with a modified data array, and copy over the old affine and header. We then use the `img.to_filename()` method to save.

In [ ]:
rescaled = ((data - data.min()) * 255. / (data.max() - data.min())).astype(np.uint8)
rescaled_img = nib.Nifti1Image(rescaled, affine=affine, header=header)

print((rescaled_img.get_data().dtype, rescaled_img.get_data_dtype()))

Our data array has the correct type, but the on-disk format is determined by the header, so saving now will not do what we want.

In [ ]:
rescaled_img.to_filename(os.path.join(out_dir, 'rescaled1.nii.gz'))
test = nib.load(os.path.join(out_dir, 'rescaled1.nii.gz'))
print((test.get_data().dtype, test.get_data_dtype()))

Instead, we need to set the data type in the header using `img.set_data_dtype()`:

In [ ]:
rescaled_img.set_data_dtype(np.uint8)
rescaled_img.to_filename(os.path.join(out_dir, 'rescaled2.nii.gz'))
test = nib.load(os.path.join(out_dir, 'rescaled2.nii.gz'))
print((test.get_data().dtype, test.get_data_dtype()))

# Check that our data is what we think it is
assert np.array_equal(test.get_data(), rescaled)

###### Aside

There are other ways to save images. The advantage to `img.to_filename()` is that, if your file extension doesn't match the image type, it won't try anything fancy; it will just fail. In my experience, a mismatch of image and file extension is much more likely to be an error than what you intended.

## Functional time series

We've explored much of nibabel's functionality (at least for NIfTI-1 images). Let's take what we've learned to look at an fMRI series and create a mean volume that could be used for alignment.

First we'll `nib-ls` the file:

In [ ]:
!nib-ls /data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-linebisection_bold.nii.gz

We see that this is a 4-dimensional dataset with 238 volumes, collected at a TR of 2.5s. We will confirm this when we load the file, and also see that the data is LAS oriented.

In [ ]:
epi = nib.load('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-linebisection_bold.nii.gz')
print(epi.get_data_dtype())
print(epi.shape)
print(epi.header.get_zooms())
print(epi.header.get_xyzt_units())
print(nib.aff2axcodes(epi.affine))

For 4D images, `img.orthoview()` also shows the time series at the voxel in focus:

In [ ]:
epi.orthoview()

Remembering that the data object is just a numpy array, we can easily take the mean over time:

In [ ]:
mean_data = epi.get_data().mean(axis=3)
mean_data.shape

In [ ]:
mean_epi = nib.Nifti1Image(mean_data, affine=epi.affine, header=epi.header)
mean_epi.orthoview()

Looks how we'd expect. Let's save it!

In [ ]:
mean_epi.to_filename(os.path.join(out_dir, 'mean_epi.nii.gz'))

## Exercise 3: 

- Display the dimensions of each functional time series for sub-01
- Plot the axial view of each mean image

In [ ]:
from glob import glob
fl = sorted(glob('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-*bold.nii.gz'))
fl

In [ ]:
fh, ax = plt.subplots(1, len(fl), figsize=(15, 5))
for idx, filename in enumerate(fl):
    img = nib.load(filename)
    print(filename, img.shape)
    ax[idx].imshow(img.get_data().mean(axis=3)[:, :, 15])    

In [ ]:
# Generate list of functional tasks

In [ ]:
# Iterate over each file and print shape, calculate mean, and display using matplotlib subplots

## Review

We've learned about nibabel, a low-level Python library for reading and writing common neuroimaging formats.

Volumetric image formats are organized in a data-affine-header structure; the data is exposed as an n-dimensional numpy array, and the orientation of the data array is encoded in both the affine and the header.

Images can be written using the `.to_filename()` method, and the on-disk data type can be adjusted with the `header.set_data_dtype()` method.

Finally, we learned about `.orthoview()` and the `nib-ls` command-line program, for rapid inspection of data files.

# Nilearn

Nilearn is a library which makes visualizing neuroimages and exposing them to machine learning algorithms straightforward.

It also simplifies a number of common tasks with neuroimages. For example, we can recreate the mean EPI image we just made in one line:

In [ ]:
img = nl.image.mean_img('/data/ds000114/sub-01/ses-test/func/sub-01_ses-test_task-linebisection_bold.nii.gz')
print(img)

Nilearn images are just nibabel images! But notice that we didn't have to copy the affine or header. Nilearn does its best to keep your data and metadata together.

Let's verify that both methods produced the same image.

In [ ]:
np.array_equal(mean_epi.get_data(), img.get_data())

In [ ]:
np.array_equal(mean_epi.affine, img.affine)

## Plotting images

Nilearn has a variety of plotting facilities. `plot_epi` shows functional images in a high-contrast color scheme.

In [ ]:
nl.plotting.plot_epi(mean_epi, cut_coords=(0, 0, 0))

In [ ]:
help(nl.plotting.plot_epi)

## Exercise 4:

Using the help output from above, redraw the mean_epi image as a set of 5 slices spanning front to back. Suppress the background using the `vmin` option.

In [ ]:
nl.plotting.plot_epi(mean_epi, display_mode='y', cut_coords=5, vmin=10)

In [ ]:
# plot the mean_epi image slices

We can also plot anatomical images. Let's show the FreeSurfer `aseg` segmentation over the T1 image we loaded earlier:

In [ ]:
nl.plotting.plot_anat(t1, dim=-1, cut_coords=(0, 0, 0))
nl.plotting.plot_roi('/data/ds000114/derivatives/freesurfer/sub-01/mri/aseg.mgz', t1,
                     dim=-1, cut_coords=(0, 0, 0))

But the overlay above looks misaligned. That's because this dataset uses a derived T1 image as input to `FreeSurfer`.

## Exercise 5:

The T1 image used for FreeSurfer is at `/data/ds000114/derivatives/fmriprep/sub-01/anat/sub-01_t1w_preproc.nii.gz`. Redraw the above plot using the correct background T1 image.

In [ ]:
t1_correct = nib.load('/data/ds000114/derivatives/fmriprep/sub-01/anat/sub-01_t1w_preproc.nii.gz')
nl.plotting.plot_anat(t1_correct, dim=-1, cut_coords=(0, 0, 0))
nl.plotting.plot_roi('/data/ds000114/derivatives/freesurfer/sub-01/mri/aseg.mgz', t1_correct,
                     dim=-1, cut_coords=(0, 0, 0))

Notice that nilearn will accept an image or a filename equally. Also recall that `t1` was a NIfTI-1 image, while `aseg` is in a FreeSurfer `.mgz` file. Nilearn takes advantage of the common interface (data-affine-header) that nibabel provides for these different formats, and makes correctly aligned overlays.

This means we can use nilearn to verify alignment, for example when testing a new algorithm.

In [ ]:
def new_algorithm(image):
    # Just mess up the affine
    bad_affine = image.affine.copy()
    bad_affine[:, :2] = mask.affine[:, 1::-1]
    return image.__class__(image.get_data(), bad_affine, mask.header)

mask = nl.image.math_img("img > 0", img='/data/ds000114/derivatives/freesurfer/sub-01/mri/brainmask.auto.mgz')
new_mask = new_algorithm(mask)

## Exercise 6:

Plot the original and messed up mask on the same background image with two colors.

In [ ]:
t1_correct = nib.load('/data/ds000114/derivatives/fmriprep/sub-01/anat/sub-01_t1w_preproc.nii.gz')
nl.plotting.plot_roi(mask, t1_correct, dim=-1, cut_coords=(0, 0, 0), cmap='Greens_r')
nl.plotting.plot_roi(new_mask, t1_correct, dim=-1, cut_coords=(0, 0, 0), cmap='Reds_r')

Nilearn can also plot results directly in MNI space using an outline. This uses the function `nl.plotting.plot_glass_brain`

In [ ]:
img = nl.image.mean_img('/data/ds000114/derivatives/fmriprep/sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold_space-mni152nlin2009casym_preproc.nii.gz')
nl.plotting.plot_glass_brain(img, threshold=1000, colorbar=True, display_mode='lyrz')

## Plotting surfaces

Nilearn has recently added surface plotting to its repertoire. Let's examine the gray/white boundary and pial surfaces.

In [ ]:
white = '/data/ds000114/derivatives/freesurfer/sub-01/surf/lh.white'
pial = '/data/ds000114/derivatives/freesurfer/sub-01/surf/lh.pial'
sulc = '/data/ds000114/derivatives/freesurfer/sub-01/surf/lh.sulc'

In [ ]:
_ = nl.plotting.plot_surf(white, bg_map=sulc)
_ = nl.plotting.plot_surf(pial, bg_map=sulc)

A common step in surface pipelines is to create a surface halfway between the white and pial surface (often called the "midthickness" or "graymid" surface). The fastest, easiest, possibly wrong (but in practice fine) way to get this surface is to take the mean of the coordinates of the corresponding vertices on the white and pial surface. We can do this straightforwardly in nibabel.

In [ ]:
wcoords, wfaces, wmeta = nib.freesurfer.read_geometry(white, read_metadata=True)
pcoords, pfaces = nib.freesurfer.read_geometry(white, read_metadata=False)

# Make sure these surfaces actually do correspond
assert np.array_equal(wfaces, pfaces)

(wcoords, wfaces, wmeta)

Notice that this is not an image object, just a tuple of coordinates, faces, and an optional metadata dictionary. And it's an example of a file nibabel doesn't handle with `nibabel.load()`.

Coordinates are the (x, y, z) coordinates of each vertex; faces are a triangle composed of three vertices, and the metadata describes the provenance and alignment.

In [ ]:
gcoords = (wcoords + pcoords) / 2
# nilearn can be pretty picky about names, so fool it into reading this as a surface file
graymid = os.path.join(out_dir, 'lh.white')
nib.freesurfer.write_geometry(graymid, gcoords, wfaces, volume_info=wmeta)

In [ ]:
_ = nl.plotting.plot_surf(graymid, bg_map=sulc, view='lateral')
_ = nl.plotting.plot_surf(graymid, bg_map=sulc, view='medial')

Looks reasonable. Let's overlay it with the `aparc` parcellation. Nilearn doesn't handle these well yet, so again, we'll load with nibabel.

In [ ]:
aparc = nib.freesurfer.read_annot('/data/ds000114/derivatives/freesurfer/sub-01/label/lh.aparc.annot')

_ = nl.plotting.plot_surf_roi(os.path.join(out_dir, 'lh.white'), aparc[0], bg_map=sulc, view='lateral')

## Exercise 7:

Plot the aparc overlay of the right hemisphere of subject 2 after calculating the mid-thickness geometry.

In [ ]:
white = '/data/ds000114/derivatives/freesurfer/sub-02/surf/rh.white'
pial = '/data/ds000114/derivatives/freesurfer/sub-02/surf/rh.pial'
sulc = '/data/ds000114/derivatives/freesurfer/sub-02/surf/rh.sulc'

wcoords, wfaces, wmeta = nib.freesurfer.read_geometry(white, read_metadata=True)
pcoords, pfaces = nib.freesurfer.read_geometry(white, read_metadata=False)

gcoords = (wcoords + pcoords) / 2
# nilearn can be pretty picky about names, so fool it into reading this as a surface file
graymid = os.path.join(out_dir, 'rh.white')
nib.freesurfer.write_geometry(graymid, gcoords, wfaces, volume_info=wmeta)

aparc = nib.freesurfer.read_annot('/data/ds000114/derivatives/freesurfer/sub-02/label/rh.aparc.annot')

_ = nl.plotting.plot_surf_roi(os.path.join(out_dir, 'rh.white'), aparc[0], bg_map=sulc, view='lateral')

In [ ]:
# Point to data

# Load data

# Calculate mid thickness

# Plot

### Review

We've explored the visualization capabilities of nilearn, which include the ability to plot BOLD images, ROIs and masks overlaid on anatomical images and surfaces. Additionally, we've used nilearn's image manipulation utilities (`mean_img`, and `math_img`) to quickly create new, valid images, and considered the dangers of destroying your affine matrix. Finally, we created our own surface using nibabel's FreeSurfer utilities.

# Conclusions

In this tutorial, we've explored loading, saving and visualizing neuroimages, as well as how nibabel and nilearn can make some more sophisticated manipulations easy. At this point, you should be able to inspect and plot most images you encounter, as well as make modifications while preserving the alignment. If we've made it through the final section, you've also seen the basic workflow for performing a wide range of statistical analyses on BOLD time series in nilearn.